In [2]:
import os,re
import pandas as pd

PROJECT="TCGA_methylation"
GENERAL="/users/genomics/marta"
DIR=os.path.join(GENERAL,PROJECT)

try:
    os.makedirs(DIR) #path where to store all the itermediate steps and outputs of the pipeline
except:
    print("Directory for %s already exists" %PROJECT)

DATASETDIR="/datasets/marta/TCGA/MET_matched_cancers_biospecimen" # path where samples are saved

GENOMEDIR="/genomics/users/marta/genomes"

try:
    os.makedirs(os.path.join(DIR,"analysis"))
    os.makedirs(os.path.join(DIR,"results"))
    #os.makedirs(os.path.join(DIR,"scripts"))
except:
    print("Directory exists")



Directory for TCGA_methylation already exists
Directory exists


In [ ]:
#### generates a file looking like normal - tumor - patient id

In [33]:
input = pd.read_csv(os.path.join(DATASETDIR,"clinical_merged_modified_bothRNAMET.tsv"))

df = input[['Project ID','File ID', 'caseID', 'Sample Type', 'gender', 'age_at_index']]
df.drop_duplicates(inplace=True)

normal = df[df['Sample Type'] == "Solid Tissue Normal"]
normal.drop(['Sample Type'], axis=1, inplace=True)
normal.rename(columns={'File ID':'normal_ID', 'caseID':'case_ID'}, inplace=True)

tumor = df[df['Sample Type'] == "Primary Tumor"]
tumor.drop(['Sample Type'], axis=1, inplace=True)
tumor.rename(columns={'File ID':'tumor_ID', 'caseID':'case_ID'}, inplace=True)

# merge
patients = pd.merge(normal, tumor, on=['Project ID','case_ID','gender','age_at_index'])
patients = patients[['Project ID','case_ID', 'normal_ID','tumor_ID', 'gender', 'age_at_index']]
patients.drop_duplicates(inplace=True)
patients.to_csv(os.path.join(DATASETDIR,"samples/patients.csv"), index=False)
patients.to_csv(os.path.join(GENERAL,PROJECT,"results/patients.csv"), index=False)

/tmp/ipykernel_4153196/3330414236.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
/tmp/ipykernel_4153196/3330414236.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal.drop(['Sample Type'], axis=1, inplace=True)
/tmp/ipykernel_4153196/3330414236.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal.rename(columns={'File ID':'normal_ID', 'caseID':'case_ID'}, inplace=True)
/tmp/ipykernel_4

Save beta values ans structure the folder knowing everytime the cancer type, the patient ID and the sample type

In [38]:
cancertypes = patients[['Project ID','case_ID']].drop_duplicates()
cancertypes_dict = cancertypes.set_index('case_ID')['Project ID'].to_dict()

normals = patients[['case_ID','normal_ID']].drop_duplicates()
normals_dict = normals.set_index('normal_ID')['case_ID'].to_dict()

tumors = patients[['case_ID','tumor_ID']].drop_duplicates()
tumors_dict = tumors.set_index('tumor_ID')['case_ID'].to_dict()

In [51]:
patients_folders = [x for x in os.listdir(os.path.join(DATASETDIR, "samples")) if os.path.isdir(os.path.join(DATASETDIR, "samples", x))]
print(len(patients_folders))
patients_folders

869


['511ad4a4-a63a-4a37-9204-95503663b5a5',
 '323ae18e-ab9c-45e0-8e27-804e1024b5ec',
 '89158820-1dc6-4fa3-92c4-a45ada0cbf03',
 'dd0309ca-341b-4277-a7d7-0ed748ced68b',
 '70ef298c-1a36-436e-8333-97dc0fc1b504',
 'cb34d57d-ac3f-44e3-93c7-c11be2a6adfd',
 '0640519d-6fb5-424c-bd52-5c3da7c96b9a',
 'b862a525-7c39-400b-bfa2-24cbed0288b2',
 '3bb64391-71a9-4881-9018-0803b4af5e60',
 'd3feed1e-9375-40aa-b866-3712e28abfc9',
 'f19c1fb0-18ce-4504-96f0-bcbd1be7793e',
 '2676a392-8cef-4a6a-89be-8c66bc40577f',
 '9128be45-bd0d-406c-9d70-db7c0ee0e31b',
 '4b2ecd36-c9d5-45f5-9113-9d2fa822a910',
 'd2910602-83ea-4adb-9ece-40f014cba060',
 '594fd047-04dc-4374-b213-eb92681b8cd6',
 '532ad62b-576b-42dd-99c7-94d07a1676e4',
 '45a42c9c-2693-4e8d-b360-2f4479ac7b70',
 '3fd6c994-40e7-4038-b23a-3cbab36e40d7',
 '6ce74f33-595c-4241-8155-50fe7cf741cd',
 'b744b337-65ff-41f3-8975-a33910c377db',
 '7d01001d-ea14-4eb1-9712-35f661a9887f',
 '317f0918-32ef-4a9b-97e3-f4636d5ce653',
 'fbf0bdc9-0f64-4f14-a815-46910d1a9755',
 '570ca02a-619d-

In [46]:
# get the cancer type and create a directory
cancer = cancertypes_dict[patientID]
try:
    os.makedirs(os.path.join(GENERAL,PROJECT,"analysis/0_bvalues",cancer))
except:
    next

In [ ]:
for folder in patients_folders:
    # know if it is a tumor or a normal sample
    if folder in normals_dict.keys():
        patientID = normals_dict[folder]
        patientID_sample = patientID + "_normal"
    elif folder in tumors_dict.keys():
        patientID = tumors_dict[folder]
        patientID_sample = patientID + "_tumor"
    else:
        next
    # get the cancer type and create a directory
    cancer = cancertypes_dict[patientID]
    print(cancer)
    print(patientID_sample)

    for file in os.listdir(os.path.join(DATASETDIR, "samples",folder)):
        if file.endswith("level3betas.txt"):
            betavalues_downloaded = pd.read_csv(os.path.join(DATASETDIR, "samples",folder,file), header=None, names=['CpG','Bvalue'], sep="\t")
    # there are some patients with more than 1 tumor sample, if that is the case, add a _2 after the patientID
    outfile = os.path.join(GENERAL,PROJECT,"analysis/0_bvalues",cancer)+"/"+patientID_sample+"_betavalues.txt"
    if os.path.isfile(outfile):
        outfile = os.path.join(GENERAL,PROJECT,"analysis/0_bvalues",cancer)+"/"+patientID_sample+"_2_betavalues.txt"
    betavalues_downloaded.to_csv(outfile, sep="\t", index=None)

TCGA-LUSC
TCGA-22-5478_tumor
TCGA-UCEC
TCGA-DI-A2QY_tumor
TCGA-KIRC
TCGA-CZ-5461_normal
TCGA-UCEC
TCGA-BK-A13C_normal
TCGA-HNSC
TCGA-CV-7103_normal
TCGA-UCEC
TCGA-E6-A1M0_normal
TCGA-KIRC
TCGA-CZ-5468_tumor
TCGA-BRCA
TCGA-BH-A1FM_tumor
TCGA-HNSC
TCGA-CV-6959_tumor
TCGA-BRCA
TCGA-BH-A18Q_tumor
TCGA-ESCA
TCGA-L5-A4OQ_tumor
TCGA-THCA
TCGA-BJ-A28R_tumor
TCGA-LIHC
TCGA-DD-A39Z_normal
TCGA-LUAD
TCGA-44-6148_normal
TCGA-KIRC
TCGA-CZ-5468_normal
TCGA-LIHC
TCGA-FV-A2QR_tumor
TCGA-BRCA
TCGA-BH-A1FM_normal
TCGA-KIRP
TCGA-BQ-7046_tumor
TCGA-BRCA
TCGA-BH-A0BW_tumor
TCGA-BRCA
TCGA-E9-A1RI_normal
TCGA-THCA
TCGA-EM-A1YC_tumor
TCGA-BRCA
TCGA-BH-A18L_tumor
TCGA-COAD
TCGA-AA-3713_normal
TCGA-THCA
TCGA-EL-A3T0_tumor
TCGA-BRCA
TCGA-BH-A18R_normal
TCGA-UCEC
TCGA-BG-A2AD_normal
TCGA-BRCA
TCGA-BH-A0DP_tumor
TCGA-ESCA
TCGA-L5-A4OQ_normal
TCGA-KIRP
TCGA-BQ-5884_normal
TCGA-COAD
TCGA-A6-2686_normal
TCGA-PRAD
TCGA-EJ-7125_tumor
TCGA-PAAD
TCGA-H6-8124_normal
TCGA-PRAD
TCGA-HC-7752_normal
TCGA-ESCA
TCGA-L5-A43C_tum

Convert Bvalues to Mvalues per cancer type

In [ ]:
%%bash -s "$GENERAL" "$PROJECT"

mkdir $1/$2/analysis/1_Mvalues

for ctype_path in $1/$2/analysis/0_bvalues/*; do
    echo $ctype_path
    cytpe=${ctype_path##*/}

    mkdir $1/$2/analysis/1_Mvalues/$ctype

    $1/$2/scripts/B2Mvalues.R $ctype_path $1/$2/analysis/1_Mvalues/$ctype